In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s3e26/sample_submission.csv
/kaggle/input/playground-series-s3e26/train.csv
/kaggle/input/playground-series-s3e26/test.csv


In [2]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Define file paths
train_path = "/kaggle/input/playground-series-s3e26/train.csv"
test_path = "/kaggle/input/playground-series-s3e26/test.csv"

# Load the training data
train_data = pd.read_csv(train_path)

# Features and target variable
features = ['N_Days', 'Drug', 'Age', 'Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema',
            'Bilirubin', 'Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides',
            'Platelets', 'Prothrombin', 'Stage']

target = 'Status'

# Ensure categorical variables are encoded consistently
label_encoder = LabelEncoder()

for col in features:
    if train_data[col].dtype == 'object':
        train_data[col] = label_encoder.fit_transform(train_data[col])

# Encode the target variable
train_data['Status'] = label_encoder.fit_transform(train_data['Status'])

# Split the data into features (X) and target variable (y)
X = train_data[features]
y = train_data['Status']

# Convert y to one-hot encoding
y = tf.keras.utils.to_categorical(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize the features
X_mean = X_train.mean(axis=0)
X_std = X_train.std(axis=0)
X_train = (X_train - X_mean) / X_std
X_val = (X_val - X_mean) / X_std

# Define the neural network model with adjusted hyperparameters
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dropout(0.5),  # Introduce dropout regularization
    tf.keras.layers.Dense(3, activation='softmax')
])

# Adjust the learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))

# Load the test data
test_path = "/kaggle/input/playground-series-s3e26/test.csv"
test_data = pd.read_csv(test_path)

# Encode categorical variables in the test set
for col in features:
    if test_data[col].dtype == 'object':
        test_data[col] = test_data[col].map(lambda s: '<unknown>' if s not in label_encoder.classes_ else s)

# Convert non-numeric columns to numeric
test_data[features] = test_data[features].apply(pd.to_numeric, errors='coerce')

# Handle missing values (NaN) in the test set
test_data[features] = test_data[features].fillna(0)  # You can replace 0 with any default value or use other imputation methods

# Normalize the test features
X_test = (test_data[features] - X_mean) / X_std

# Make predictions on the test set
predictions = model.predict(X_test)

Epoch 1/20
198/198 [==============================] - 1s 4ms/step - loss: 0.6220 - accuracy: 0.7617 - val_loss: 0.5324 - val_accuracy: 0.7957
Epoch 2/20
198/198 [==============================] - 1s 3ms/step - loss: 0.5473 - accuracy: 0.7892 - val_loss: 0.5175 - val_accuracy: 0.8065
Epoch 3/20
198/198 [==============================] - 1s 4ms/step - loss: 0.5304 - accuracy: 0.7955 - val_loss: 0.5130 - val_accuracy: 0.8071
Epoch 4/20
198/198 [==============================] - 1s 3ms/step - loss: 0.5182 - accuracy: 0.8015 - val_loss: 0.5084 - val_accuracy: 0.8102
Epoch 5/20
198/198 [==============================] - 1s 3ms/step - loss: 0.5117 - accuracy: 0.8047 - val_loss: 0.5033 - val_accuracy: 0.8090
Epoch 6/20
198/198 [==============================] - 1s 3ms/step - loss: 0.5058 - accuracy: 0.8028 - val_loss: 0.5086 - val_accuracy: 0.8102
Epoch 7/20
198/198 [==============================] - 1s 3ms/step - loss: 0.5012 - accuracy: 0.8071 - val_loss: 0.5053 - val_accuracy: 0.8128
Epoch 